In [1]:
import pandas as pd

In [ ]:
data_2017 = pd.read_csv('../data/school_based/2017.csv')
data_2018 =pd.read_csv('../data/school_based/2018.csv')
data_2019 = pd.read_csv('../data/school_based/2019.csv')
data_2021 = pd.read_csv('../data/school_based/2021.csv')

2017 does not have the school_name and system_name columns.  I will create a mapping from the 2018 dataframe to backfill the missing strings

In [ ]:
# Create a unique mapping of 2018 dataframe
mapping_2018 = data_2018.drop_duplicates(subset=['system', 'school'])

# Merge the dataframes
data_2017 = pd.merge(data_2017, mapping_2018[['system', 'system_name', 'school', 'school_name']], on=['system', 'school'], how='left')

# Specify column order to match subsequent dataframes
new_order = ['year', 'system', 'system_name', 'school', 'school_name', 'subject', 'grade', 'subgroup', 'valid_tests',
             'n_below', 'n_approaching', 'n_on_track', 'n_mastered', 'pct_below', 
             'pct_approaching', 'pct_on_track', 'pct_mastered', 'pct_on_mastered']

# Reorder the columns
data_2017 = data_2017[new_order]

In [ ]:
data_2021 = pd.read_csv('../data/school_based/2021.csv')
data_2021.info()

In [ ]:
data_2022 = pd.read_excel('../data/school_based/2022.xlsx')
data_2022.to_csv (r'../data/school_based/2022.csv', index = None, header=True)

In [ ]:
data_2019.head(n=1)

In [ ]:
data_2021.head(n=1)